In [4]:
import sys
import getpass
sys.path.append('..')

In [5]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from resources.models import Surveys, Forms, QuestionGroups, Questions, SurveyInstances, Answers, Sync
from resources.api import flow_api, flow_sync
from resources.utils import marktime, checktime, answer_handler
from resources.database import write_data, clear_schema, schema_generator
from resources.connection import engine_url
from collections import ChainMap
import pandas as pd
from pandas.io import sql

In [6]:
instance_name = 'uat2'
api = flow_api()
sync = flow_sync()
request_url = api.get_instance_url(instance_name)
engine_url = engine_url()
engine = create_engine(engine_url)
session = sessionmaker(bind=engine)()
getter = sessionmaker(bind=engine)()

## Populating Survey

In [ ]:
surveysUrl = []
token = api.get_new_token()
start_time = marktime()

In [ ]:
def getFolders(items, token):
    for folder in items['folders']:
        try:
            surveysUrl.append(folder['surveysUrl'])
            token = api.check_token(token)
            childs = api.get_data(folder['foldersUrl'], token)
            getFolders(childs, token)
        except:
            pass

print('GETTING FOLDER LIST: ' + checktime(start_time))
token = api.check_token(token)
parents = api.get_data(request_url + '/folders', token)
getFolders(parents, token)
print('FOLDER IS POPULATED: ' + checktime(start_time))

surveys = []
print('GETTING SURVEY LIST: ' + checktime(start_time))
for surveyUrl in surveysUrl:
    token = api.check_token(token)
    surveyList = api.get_data(surveyUrl, token)
    for survey in surveyList['surveys']:
        surveys.append(survey)
print('SURVEY IS POPULATED: ' + checktime(start_time))

## Recording Surveys

In [ ]:
print('RECORDING SURVEY: ' + checktime(start_time))
formInstanceUrls = []
for url in surveys:
    token = api.check_token(token)
    data = api.get_data(url['surveyUrl'], token)
    print('GETTING {}: {}'.format(data['name'],checktime(start_time)))
    if data['registrationFormId'] == "":
        data.update({'registrationFormId':0})
    input_data = Surveys(data)
    write_data(session, input_data, data, "SURVEY")
    for form in data['forms']:
        form.update({'survey_id':data['id']})
        input_data = Forms(form)
        write_data(session, input_data, form, False)
        for qgroup in form['questionGroups']:
            qgroup.update({'form_id':form['id']})
            input_data = QuestionGroups(qgroup)
            write_data(session, input_data, qgroup, False)
            for question in qgroup['questions']:
                question.update({'group_id':qgroup['id']})
                question.update({'form_id':form['id']})
                input_data = Questions(question)
                write_data(session, input_data, question, False)
        formInstanceUrls.append({
            'formInstancesUrl': form['formInstancesUrl'],
            'form_id': form['id']
        })
print('SURVEY IS RECORDED: ' + checktime(start_time))

## Recording Data

In [ ]:
def saveAnswers(group, index):
    for qid in [*group]:
        question = getter.query(Questions).filter(Questions.id == int(qid)).first()
        answer = {
            'survey_instance_id': instance['id'],
            'question_id': qid,
            'repeat_index': index
        }
        answer_value = ''
        if group[qid]:
            answer_value = answer_handler(group, qid, question.type)
            answer_value = str(answer_value)
        answer.update({'value':answer_value})
        input_data = Answers(answer)
        write_data(session, input_data, answer, False)

In [ ]:
print('GETTING SURVEY INSTANCES: ' + checktime(start_time))
for data in formInstanceUrls:
    token = api.check_token(token)
    formInstances = api.get_data(data['formInstancesUrl'], token)
    formInstancesData = formInstances['formInstances']
    while 'nextPageUrl' in formInstances:
        token = api.check_token(token)
        nextPageData = api.get_data(formInstances['nextPageUrl'], token)
        formInstancesData += nextPageData['formInstances']
        formInstances = nextPageData
    for instance in formInstancesData:
        input_data = SurveyInstances(instance)
        write_data(session, input_data, instance, "SURVEY INSTANCES")
        answers = instance['responses']
        if answers is not None:
            for group_id in [*answers]:
                saved_group = getter.query(QuestionGroups).filter(QuestionGroups.id == group_id).first()
                saved_questions = getter.query(Questions).filter(Questions.question_group_id == group_id).all()
                for index, group in enumerate(answers[group_id], start=1):
                    if saved_group.repeat == False:
                        index = 0
                    if saved_group.repeat:
                        for saved_question in saved_questions:
                            group.update({saved_question.id: None})
                    print(group)
                    saveAnswers(group, index)
print('SURVEY INSTANCES RECORDED: ' + checktime(start_time))

## Generate Required Tables & Init Sync

In [ ]:
clear_schema(engine)
schema_generator(session, engine)
initial_state = sync.init_data(session, instance_name, token)

## Sync Tests

In [7]:
next_sync = sync.cursor_get(session)
token = sync.get_token()
new_data = sync.get_data(session, next_sync, token)

CURSOR UPDATED
NEW CURSOR SAVED


In [8]:
## Test History
new_data
sync_history = sync.get_history(session)

In [9]:
new_data

{'changes': {'dataPointDeleted': ['215100003'],
  'dataPointChanged': [],
  'formChanged': [],
  'surveyChanged': [],
  'formInstanceDeleted': ['217120009'],
  'formInstanceChanged': [],
  'formDeleted': [],
  'surveyDeleted': []},
 'nextSyncUrl': 'https://api-auth0.akvotest.org/flow/orgs/uat2/sync?next=true&cursor=6711',
 'status': 200}

In [ ]:
api_token = api.get_new_token()
start_time = marktime()

### Remove Data

In [ ]:
survey_instance_deleted = sync_history.data['dataPointDeleted']
form_deleted = sync_history.data['formDeleted']
survey_deleted = sync_history.data['surveyDeleted']

### Update Data

In [ ]:
survey_changed = sync_history.data['surveyChanged']
form_changed = sync_history.data['formChanged']
survey_instance_changed = sync_history.data['formInstanceChanged']
formInstanceUrls = []

#### Update Survey & Forms

In [ ]:
def check_folders(foldersUrl, surveysUrl, api_token):
    folder_depth = api.get_data(foldersUrl, api_token)['folders']
    if len(check_folder) > 0:
        for df in folder_depth:
            surveyUrl.append(df['surveysUrl'])
            check_folders(df, surveyUrl, api_token)
    else:
        return surveysUrl

api_token = api.check_token(api_token)
folders = api.get_data(request_url + '/folders', api_token)
new_surveys = []
for sc in survey_changed:
    for folder in folders['folders']:
        survey_url_check = []
        if folder['id'] == sc['id']:
            api_token = api.check_token(api_token)
            survey_collections = check_folders(folder['foldersUrl'], folder['surveysUrl'], api_token)
            survey_url_check.append(survey_collections)
        for survey_url in survey_url_check:
            api_token = api.check_token(api_token)
            surveys = api.get_data(survey_url, api_token)
            for survey in surveys['surveys']:
                new_surveys.append(survey)
    for survey in new_surveys:
        if survey['id'] == sc['id']:
            data = api.get_data(survey['surveyUrl'], api_token)
            for form in data['forms']:
                form.update({'survey_id':data['id']})
                stored_form = session.query(Forms).filter(Forms.id == int(form['id'])).first()
                if stored_form == None:
                    input_data = Forms(form)
                    write_data(session, input_data, form, False)
                for qgroup in form['questionGroups']:
                    qgroup.update({'form_id':form['id']})
                    stored_qgroup = session.query(QuestionGroups).filter(QuestionGroups.id == int(qgroup['id'])).first()
                    if stored_qgroup == None:
                        input_data = QuestionGroups(qgroup)
                        write_data(session, input_data, qgroup, False)
                    for question in qgroup['questions']:
                        stored_question = session.query(Questions).filter(Questions.id == int(question['id'])).first()
                        question.update({'group_id':qgroup['id']})
                        question.update({'form_id':form['id']})
                        if stored_question == None:
                            input_data = Questions(question)
                            write_data(session, input_data, question, False)
                    formInstanceUrls.append({
                        'formInstancesUrl': form['formInstancesUrl'],
                        'form_id': form['id']
                    })
            stored_survey = session.query(Surveys).filter(Surveys.id == int(data['id'])).first()
            print('GETTING {}: {}'.format(data['name'],checktime(start_time)))
            if data['registrationFormId'] == "":
                    data.update({'registrationFormId':0})
            if stored_survey:
                stored_survey.registration_id = data['registrationFormId']
                stored_survey.name = data['name']
                print('SURVEY UPDATED: {}'.format(data['name']))
                session.add(stored_survey)
                session.commit()
            else:
                input_data = Surveys(data)
                print('NEW SURVEY FOUND: {}'.format(data['name']))
                write_data(session, input_data, data, "SURVEY")

In [ ]:
print('GETTING SURVEY INSTANCES: ' + checktime(start_time))
for instance in survey_instance_changed:
    stored_instance = session.query(SurveyInstances).filter(SurveyInstances.id == instance['id']).first()
    if stored_instance == None:
        input_data = SurveyInstances(instance)
        write_data(session, input_data, instance, "SURVEY INSTANCES")
    answers = instance['responses']
    if answers is not None:
        stored_answers = session.query(Answers).filter(Answers.survey_instance_id == instance['id'])
        stored_answers.delete(synchronize_session='evaluate')
        for group_id in [*answers]:
            saved_group = getter.query(QuestionGroups).filter(QuestionGroups.id == group_id).first()
            saved_questions = getter.query(Questions).filter(Questions.question_group_id == group_id).all()
            for index, group in enumerate(answers[group_id], start=1):
                if saved_group.repeat == False:
                    index = 0
                if saved_group.repeat:
                    for saved_question in saved_questions:
                        group.update({saved_question.id: None})
                saveAnswers(group, index)
print('SURVEY INSTANCES RECORDED: ' + checktime(start_time))

In [ ]:
clear_schema(engine)
schema_generator(session, engine)

#### Update Forms